## Data for Doodle Scheduling problem

·The decision variables are the whether a staff member attend this meeting at certain time periods.

·For each person, he/she can only attend one meeting during the whole day.

·For time slots except the first and lunch, there could only be one person meeting the candidate. For the first slot, there is supposed to be one person together with either Mirjam or Matt, while for the lunch slot, there will be three persons

·The objective is to find a proper arrangement. Here we can just set the objective is to minimize the sum of the decision variables.

Math model:

Objective -> minimize $\sum_{j=1}^{13}\sum_{i=1}^{16} c_{ij}$, $c_i$ is whether the person show up at this time period. If $c_{ij} = 0$, he/she will not, but he/she will if $c_i = 1$.

Constraints -> $c_{ij} \in \{1,0\}$; $\sum_{j=1}^{13} c_ij = 1$ ; $\sum_{i=1}^{16} c_ij = 1$ when $j \neq 1 or 7$; $\sum_{i=1}^{16} c_i1 = 2 \land c_11 + c_21 =1$; $\sum_{i=1}^{16} c_i7 = 3$

In [2]:
#using Pkg
#Pkg.add("NamedArrays")

using JuMP, NamedArrays

availability =
  [ 1 1 0 0 0 0 0 0 0 0 1 1 1
    1 1 1 0 0 0 0 0 0 1 1 0 0
    0 0 1 1 0 0 0 1 1 0 0 0 0
    0 1 1 0 0 0 0 0 1 1 0 0 0
    0 0 0 1 1 0 1 1 0 1 1 1 1
    0 0 0 1 1 1 1 1 1 1 1 1 0
    0 0 0 0 0 0 1 1 1 0 0 0 0
    0 1 1 0 0 0 0 0 1 1 0 0 0
    0 0 0 1 1 1 1 0 0 0 0 0 0
    0 0 0 0 0 0 0 1 1 0 0 0 0
    0 0 0 0 0 0 1 1 1 0 0 0 0
    1 1 0 0 0 1 1 1 1 0 0 1 1
    1 1 1 0 1 1 0 0 0 0 0 1 1
    0 1 1 1 0 0 0 0 0 0 0 0 0
    0 1 1 0 0 0 0 1 1 1 0 0 0
    1 1 0 0 1 1 0 0 0 0 0 0 0 ]

TIMES = collect(1:13)
NAMES = [:Mirjam,:Matt,:Manuel,:Luca,:Jule,:Michael,:Malte,:Chris,:Spyros,:Florian,:Josep,:Joel,:Tom,:Daniel,:Christian,:Anne ]
TIMESTR = ["10:00","10:20","10:40","11:00","11:20","11:40","lunch","1:00","1:20","1:40","2:00","2:20","2:40"]

# Creating a NamedArray of the availability data
times = NamedArray( availability, (NAMES,TIMES), ("NAME","TIME"))
#println(times)

## With a NamedArray, it is possible to use symbols as indices (although you don't have to use this)
## For example, it is possible to write

#println(times[:Luca,1])

# or

#println(sum(times[i,1] for i in NAMES))


# Yizhou Code starts
rows, cols = size(availability);

using Clp
m = Model(with_optimizer(Clp.Optimizer))
@variable(m, 0 <= x[1:16,1:13] <= 1)
for i = 1:rows
    for j = 1:cols
        if availability[i,j] == 0
            @constraint(m,x[i,j] == 0);
        end
    end
    @constraint(m,sum(x[i,j] for j = 1:cols ) == 1);
end

for b = 1:13
    if b == 7
        @constraint(m, sum(x[a,b] for a = 1:rows) == 3);
    elseif b == 1
        @constraint(m,x[1,b]+ x[2,b] == 1);
        @constraint(m,sum(x[a,b] for a = 1:rows) == 2);
    else
        @constraint(m, sum(x[a,b] for a = 1:rows) == 1);
    end
end

@objective(m, Min, sum(x));
optimize!(m);


schedule = zeros(Int64,rows,cols); # Very important to declare "Int" here
for i = 1:rows
    for j = 1:cols        
        schedule[i,j] = (JuMP.value(x[i,j]));              
    end
end

times2 = NamedArray((schedule),(NAMES,TIMES),("Name","Time"))

println((times2))

16×13 Named Array{Int64,2}
Name ╲ Time │  1   2   3   4   5   6   7   8   9  10  11  12  13
────────────┼───────────────────────────────────────────────────
:Mirjam     │  0   0   0   0   0   0   0   0   0   0   1   0   0
:Matt       │  1   0   0   0   0   0   0   0   0   0   0   0   0
:Manuel     │  0   0   0   0   0   0   0   0   1   0   0   0   0
:Luca       │  0   0   1   0   0   0   0   0   0   0   0   0   0
:Jule       │  0   0   0   0   0   0   0   0   0   0   0   1   0
:Michael    │  0   0   0   0   0   1   0   0   0   0   0   0   0
:Malte      │  0   0   0   0   0   0   1   0   0   0   0   0   0
:Chris      │  0   1   0   0   0   0   0   0   0   0   0   0   0
:Spyros     │  0   0   0   0   1   0   0   0   0   0   0   0   0
:Florian    │  0   0   0   0   0   0   0   1   0   0   0   0   0
:Josep      │  0   0   0   0   0   0   1   0   0   0   0   0   0
:Joel       │  0   0   0   0   0   0   1   0   0   0   0   0   0
:Tom        │  0   0   0   0   0   0   0   0   0   0   0   0   

In [3]:
availability - schedule

16×13 Array{Int64,2}:
 1  1  0  0  0  0  0  0  0  0  0  1  1
 0  1  1  0  0  0  0  0  0  1  1  0  0
 0  0  1  1  0  0  0  1  0  0  0  0  0
 0  1  0  0  0  0  0  0  1  1  0  0  0
 0  0  0  1  1  0  1  1  0  1  1  0  1
 0  0  0  1  1  0  1  1  1  1  1  1  0
 0  0  0  0  0  0  0  1  1  0  0  0  0
 0  0  1  0  0  0  0  0  1  1  0  0  0
 0  0  0  1  0  1  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  1  1  0  0  0  0
 1  1  0  0  0  1  0  1  1  0  0  1  1
 1  1  1  0  1  1  0  0  0  0  0  1  0
 0  1  1  0  0  0  0  0  0  0  0  0  0
 0  1  1  0  0  0  0  1  1  0  0  0  0
 0  1  0  0  1  1  0  0  0  0  0  0  0

## Data for stadium building problem

In [2]:
using JuMP

tasks = 1:18
durations = [2 16 9 8 10 6 2 2 9 5 3 2 1 7 4 3 9 1]
predecessors = ( [], [1], [2], [2], [3], [4,5], [4], [6], [4,6], [4], [6], [9], [7], [2], [4,14], [8,11,14], [12], [17] )
pred_dict = Dict(zip(tasks,predecessors));   # dictionary mapping tasks --> predecessors.

# additional columns of data (maximum reduction possible )
max_reduction =  [0,  3,  1,  2,  2,  1, 1, 0,  2,  1,  1, 0, 0,  2,  2, 1,  3, 0]  # max reduction (in weeks)
cost_reduction = [0, 30, 26, 12, 17, 15, 8, 0, 42, 21, 18, 0, 0, 22, 12, 6, 16, 0]  # cost of reduction ($1,000/week)
bonus_amount = 30     # bonus for expediting the project ($1,000/week )
;

-------------------------------------------------------------------------------------------------------------------
a).

In [3]:
m = Model(with_optimizer(Clp.Optimizer));
@variable(m, tstart[tasks]);

for i in tasks
    for j in pred_dict[i]
        @constraint(m, tstart[i] >= tstart[j] + durations[j]);
    end
end

@constraint(m, tstart[1] == 0);
@objective(m, Min, tstart[18] + durations[18])

optimize!(m)
println(value.(tstart))
println("minimum duration: ", objective_value(m))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:18
And data, a 18-element Array{Float64,1}:
  0.0
  2.0
 18.0
 18.0
 27.0
 37.0
 26.0
 43.0
 43.0
 26.0
 43.0
 52.0
 28.0
 18.0
 26.0
 46.0
 54.0
 63.0
minimum duration: 64.0
Coin0506I Presolve 0 (-23) rows, 0 (-18) columns and 0 (-45) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 64
Coin0511I After Postsolve, objective 64, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 64 - 0 iterations time 0.002, Presolve 0.00


ANSWER: The earlist accomplishment of this construction will be achieved in 64 weeks.

----------------------------------------------------------------------------------------------------------------------
b).

The objective is to minimize the cost, which is the bonus minus the additional cost for shortening the schdule. In the following code, the shortened time period is represented as tred[] (Time REDucation). By adding this variables, the previous model can be modified to achieve the new objective.

In [4]:
m = Model(with_optimizer(Clp.Optimizer));

@variable(m, tstart[tasks]);
@variable(m, tred[tasks] >= 0);



for i in tasks
    @constraint(m, tred[i] <= max_reduction[i])
    for j in pred_dict[i]        
        @constraint(m, tstart[i] >= tstart[j] + durations[j] - tred[j]);
    end
end

@constraint(m, tstart[1] == 0);

@objective(m, Max, (64 - (tstart[18] + durations[18])) * 30 - sum((cost_reduction[i] * tred[i]) for i in tasks));
optimize!(m)
println(value.(tstart))
println("end time -> ", value.(tstart[18]+durations[18]), " weeks")
println("maximum bonus -> ", objective_value(m), "k")

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:18
And data, a 18-element Array{Float64,1}:
  0.0
  2.0
 15.0
 15.0
 23.0
 31.0
 23.0
 36.0
 36.0
 23.0
 36.0
 45.0
 25.0
 15.0
 23.0
 39.0
 47.0
 53.0
end time -> 54.0 weeks
maximum bonus -> 87.0k
Coin0506I Presolve 3 (-38) rows, 8 (-28) columns and 18 (-67) elements
Clp0006I 0  Obj 363 Primal inf 13.199998 (2)
Clp0006I 2  Obj 87
Clp0000I Optimal - objective value 87
Coin0511I After Postsolve, objective 87, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 87 - 2 iterations time 0.002, Presolve 0.00


ANSWER: So the end time is in 54 weeks so that the profit is maximized.

## Museum site planning.

In [5]:
#using Pkg
#Pkg.add("LinearAlgebra")

using Clp,JuMP,LinearAlgebra;
A = [ -1 0; 0 -1; 0 1; 2/3 1; 3 -1];
b = [ 0  ; 0  ; 500; 700; 1500];

In [6]:
m = Model(with_optimizer(Clp.Optimizer))
@variable(m, r >= 0); # radius
@variable(m, x[1:2]); # center

B = A * x;
clearance = 50;
for i = 1:length(A[:,1])
     @constraint(m, B[i] + (r+clearance)*norm(A[i,:]) <= b[i])
end
@objective(m, Max, r);
optimize!(m)

Cen = value.(x);
Rad = value(r);



Coin0506I Presolve 4 (-1) rows, 2 (-1) columns and 8 (-4) elements
Clp0006I 0  Obj -0 Primal inf 0.099999 (1) Dual inf 0.40283601 (1)
Clp0006I 2  Obj 194.02853
Clp0000I Optimal - objective value 194.02853
Coin0511I After Postsolve, objective 194.02853, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 194.0285268 - 2 iterations time 0.002, Presolve 0.00


In [7]:
println("The coordinate of center is ->", Cen );
println("The radius is ->",Rad);

The coordinate of center is ->[244.02852679380186, 244.02852679380186]
The radius is ->194.02852679380186


ANSWER: The center is located at (244.02852679380186, 244.02852679380186) on this map, and the radius is 194.02852679380186.

In [8]:
using Plots

In [ ]:

Raw = [0 0; 0 500; 300 500; 600 300; 500 0; 0 0];
outline = plot(Raw[:,1], Raw[:,2], label = "Neighbourhood",color = "blue");

L = 101;
theta = [i for i = 0: (L-1)] * (2*pi)/(L-1);
X_opt = zeros(L,1);
Y_opt = zeros(L,1);
for i = 1:length(theta)
    X_opt[i] = Cen[1] + Rad * cos(theta[i]);
    Y_opt[i] = Cen[2] + Rad * sin(theta[i]);
end

plot!(outline,X_opt,Y_opt,label = "Museum",color = "red")
